# Heat Exchanger Analysis: The Effectiveness–NTU Method
Hot exhaust gases, which enter a finned-tube, cross-flow heat exchanger at 300<sup>o</sup>C and
leave at 100<sup>o</sup>C, are used to heat pressurized water at a flow rate of 1 kg/s from 35 to
125<sup>o</sup>C. The overall heat transfer coefficient based on the gas-side surface area is
$U_h$=100W/m<sup>2</sup>K. Determine the required gas-side surface area $A_h$ using the NTU method.

In [1]:
## Known

In [2]:
from functools import partial
import numpy as np
from scipy.optimize import root

from thermal_system_calculation import to_kelvin

temp_gas_in = to_kelvin(300) # kelvin
temp_gas_out = to_kelvin(100) # kelvin
mass_flow_water = 1 # kg/s
temp_water_in = to_kelvin(35) #kelvin
temp_water_out = to_kelvin(125) # kelvin
u_h = 100 # W/m2K

In [3]:
## Find
area_gas_side = None

## Schematic
![Schematic](images/heat_exchanger_effectiveness_NTU.png)

## Assumptions
1. Negligible heat loss to the surrounding and kinetic and potential energy changes
2. Constant properties
3. Steady state

## Properties

In [4]:
c_p_water = 4197 # J/kgK

## Analysis
In this analysis, we know the temperature input and output of both mediums. However, we cannot
use the logarithmic mead temperature difference because it is cross-flow. Therefore, we will use
the definition of effectiveness,

$$\epsilon = \frac{q}{q_\max}$$

, to find its value and find the value of NTU from the correlation
given. Then, we can find the area from the definition of NTU.

$$NTU = \frac{UA}{C_\min}$$

First, the heat transfer rate can be calculated using the energy balance.

In [5]:
q = mass_flow_water * c_p_water * (temp_water_out - temp_water_in)
print(f"Heat transfer rate is {q} W")

Heat transfer rate is 377730.0 W


Heat capacity rate for the water and gas can be calculated.

In [6]:
heat_capacity_rate_water = mass_flow_water * c_p_water
heat_capacity_rate_gas = q / (temp_gas_in - temp_gas_out)
heat_capacity_rate_min = min([heat_capacity_rate_gas, heat_capacity_rate_water])
heat_capacity_rate_max = max([heat_capacity_rate_gas, heat_capacity_rate_water])

print(f"Minimum heat capacity rate is {heat_capacity_rate_min:.2f}W/K")

Minimum heat capacity rate is 1888.65W/K


Now we will calculate the effectiveness and $C_\min / C_\max$ to find the value of NTU from the
correlation.

In [7]:
q_max = heat_capacity_rate_min * (temp_gas_in - temp_water_in)
effectiveness = q / q_max
heat_capacity_rate_ratio = heat_capacity_rate_min / heat_capacity_rate_max

Now we can find NTU value solving the equation,

$$\epsilon = 1 - \exp\left[\frac{1}{C_r}NTU^{0.22}\left\{\exp\left[-C_r(NTU)^{0.78} \right] - 1\right\}\right]$$

In [8]:
def get_effectiveness_cross_flow_unmixed(ntu: float, c_r: float) -> float:
    """Returns effectiveness of heat exchanger of unmixed cross flow"""
    return 1 - np.exp(1 / c_r * ntu ** 0.22 * (np.exp(-c_r * ntu ** 0.78) - 1))

def equation_to_solve(ntu) -> float:
    return effectiveness - get_effectiveness_cross_flow_unmixed(ntu, heat_capacity_rate_ratio)

ntu0 = u_h * 1 / heat_capacity_rate_min
solution = root(equation_to_solve, [ntu0])
ntu = solution.x[0]

print(f"NTU for the given condition is  {ntu}")

NTU for the given condition is  2.023870529497853


Finally, the area of heat exchanger is found.py

In [9]:
area_gas_side = ntu * heat_capacity_rate_min / u_h
print(f"The area of heat exchanger is {area_gas_side} m2")


The area of heat exchanger is 38.223830755361206 m2
